## Preparation

In [ ]:
# setup
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 19.7MB/s 


### Import the libraries

In [ ]:
import os
import tempfile
import numpy as np
import pandas as pd
import pickle
import json
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# use tensorflow
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# use tensorflow model optimization
import tensorflow_model_optimization as tfmot

# mount the Google Drive to Google Colab
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

%cd "/content/drive/My Drive/"

%load_ext tensorboard

Found GPU at: /device:GPU:0


### Load data

In [ ]:
f = open('accgry_segments.pckl', 'rb')
segments = pickle.load(f)
f.close()

f = open('accgry_labels.pckl', 'rb')
labels = pickle.load(f)
f.close()

In [ ]:
# config based on data preprocessing
sampling_freq = 100
window_size = int(5.12*sampling_freq)
overlap = 1*sampling_freq # 1s overlap
feature_size = 18
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
reshaped_segments = segments.reshape(len(segments),window_size, feature_size)

### Split data

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(reshaped_segments, labels, test_size=0.1, random_state=1)

### Load the trained model

In [ ]:
# load the trained lstm model
lstm = tf.keras.models.load_model('lstm')
# check the model architecture
lstm.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 512, 100)          47600     
_________________________________________________________________
dropout_5 (Dropout)          (None, 512, 100)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 12)                1212      
Total params: 139,312
Trainable params: 139,312
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# evaluate model
_, lstm_accuracy = lstm.evaluate(test_x, test_y, verbose=0)
lstm_accuracy

0.9826689958572388

## Fine-tune pre-trained model with pruning


### Define the model

In [ ]:
# start the model with 50% sparsity (50% zeros in weights) and end with 80% sparsity.
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# compute end step to finish pruning after 10 epochs.
batch_size = 64
epochs = 15
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_features = train_x.shape[0] * (1 - validation_split)
end_step = np.ceil(num_features / batch_size).astype(np.int32) * epochs

# define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(lstm, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_lstm_7 ( (None, 512, 100)          94803     
_________________________________________________________________
prune_low_magnitude_dropout_ (None, 512, 100)          1         
_________________________________________________________________
prune_low_magnitude_lstm_8 ( (None, 100)               160403    
_________________________________________________________________
prune_low_magnitude_dense_8  (None, 100)               20102     
_________________________________________________________________
prune_low_magnitude_dense_9  (None, 12)                2414      
Total params: 277,723
Trainable params: 139,312
Non-trainable params: 138,411
_________________________________________________________________


### Prun the model

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
]
  
model_for_pruning.fit(train_x, train_y,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

model_for_pruning = tf.keras.models.load_model('lstm_pruning')

Epoch 1/15
219/219 [==============================] - 19s 68ms/step - loss: 0.1060 - accuracy: 0.9681 - val_loss: 0.1352 - val_accuracy: 0.9602
Epoch 2/15
219/219 [==============================] - 13s 61ms/step - loss: 0.0850 - accuracy: 0.9775 - val_loss: 0.1009 - val_accuracy: 0.9724
Epoch 3/15
219/219 [==============================] - 13s 62ms/step - loss: 0.0706 - accuracy: 0.9786 - val_loss: 0.1202 - val_accuracy: 0.9641
Epoch 4/15
219/219 [==============================] - 13s 61ms/step - loss: 0.0743 - accuracy: 0.9772 - val_loss: 0.0937 - val_accuracy: 0.9705
Epoch 5/15
219/219 [==============================] - 13s 62ms/step - loss: 0.0725 - accuracy: 0.9797 - val_loss: 0.0793 - val_accuracy: 0.9750
Epoch 6/15
219/219 [==============================] - 13s 60ms/step - loss: 0.0789 - accuracy: 0.9757 - val_loss: 0.0949 - val_accuracy: 0.9673
Epoch 7/15
219/219 [==============================] - 14s 64ms/step - loss: 0.0554 - accuracy: 0.9830 - val_loss: 0.1080 - val_accuracy:

### Summarize the pruned model

In [ ]:
# summarize the pruned model architecture
model_for_pruning.summary()

# evaluate the pruned model 
_, model_for_pruning_accuracy = model_for_pruning.evaluate(test_x, test_y, verbose=0)

# compare the pruned model with baseline
print('Baseline test accuracy:',lstm_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_lstm_7 ( (None, 512, 100)          87602     
_________________________________________________________________
prune_low_magnitude_dropout_ (None, 512, 100)          1         
_________________________________________________________________
prune_low_magnitude_lstm_8 ( (None, 100)               120402    
_________________________________________________________________
prune_low_magnitude_dense_8  (None, 100)               20102     
_________________________________________________________________
prune_low_magnitude_dense_9  (None, 12)                2414      
Total params: 230,521
Trainable params: 139,312
Non-trainable params: 91,209
_________________________________________________________________
Baseline test accuracy: 0.9826689958572388
Pruned test accuracy: 0.9809358716011047


### Create 3x smaller models from pruning

In [ ]:
def get_gzipped_model_size(file):
  # returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
# first, create a compressible model for TensorFlow
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(lstm, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved baseline model to: /tmp/tmpab52pzt3.h5
Saved pruned Keras model to: /tmp/tmprdtauiq_.h5


In [ ]:
# apply a standard compression algorithm (e.g. via gzip)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))

Size of gzipped baseline Keras model: 520224.00 bytes
Size of gzipped pruned Keras model: 193012.00 bytes
